In [31]:
import os
import cv2
from PIL import Image
import pickle
import sys
import dlib
import numpy as np
from skimage import io
from math import sqrt
from csv import writer, reader
from sklearn.preprocessing import StandardScaler
import pandas as pd
import face_recognition


#-------------------------------------------------------------------------------
def compare(data):
    min=999999999999
    with open('compare.csv', "r") as file:
        csv_reader = reader(file)
        for row in csv_reader:
            #print(row)
            sum=0
            print('comparing.................')
            for i in range (1,len(row)):
                x=float(data[i][0])
                sum = sum + (float(row[i])-x)**2
            print(row[0],"  ",sum)
            if(sum < min):
                min=sum
                name=row[0]
    print("Recognised as :"+name)
                
#-------------------------------------------------------------------------------
def facecompare(image):
    predictor_model = "shape_predictor_68_face_landmarks.dat"

    # Create a HOG face detector using the built-in dlib class
    face_detector = dlib.get_frontal_face_detector()
    face_pose_predictor = dlib.shape_predictor(predictor_model)


    # Run the HOG face detector on the image data
    detected_faces = face_detector(image, 1)

    print("Found {} faces.".format(len(detected_faces)))

    # to get the window to show image
    win = dlib.image_window()
    # Show the desktop window with the image
    win.set_image(image)



    # Loop through each face we found in the image
    for i, face_rect in enumerate(detected_faces):

        # Detected faces are returned as an object with the coordinates 
        # of the top, left, right and bottom edges
        print("- Face #{} found at Left: {} Top: {} Right: {} Bottom: {}".format(i, face_rect.left(), face_rect.top(), face_rect.right(), face_rect.bottom()))

        # Draw a box around each face we found
        win.add_overlay(face_rect)
        # Get the the face's pose
        pose_landmarks = face_pose_predictor(image, face_rect)
        # Draw the face landmarks on the screen.
        win.add_overlay(pose_landmarks)
        
        vec = np.empty([68, 2], dtype = int)
        for b in range(68):
            vec[b][0] = pose_landmarks.part(b).x
            vec[b][1] = pose_landmarks.part(b).y
        print("..........................")


        dist = []
        for i in range(68):
            for j in range(i+1,68):
                dist.append(sqrt((vec[i][0]-vec[j][0])**2 + (vec[i][1]-vec[j][1])**2))
        #print(dist)
        dist = list(map(lambda x: [x], dist))
        dataset = (dist)
        sc_dist = StandardScaler()
        sc_dist = sc_dist.fit_transform(dataset)
        print(sc_dist)
        compare(sc_dist)
#--------------------------------------------------------------------------------


video_cap = cv2.VideoCapture(0)
while True:
    ret, frame = video_cap.read()
    cv2.imshow('Input', frame)
    facecompare(frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'): break
	
video_cap.release()
cv2.destroyAllWindows()
		
		
		
